In [4]:
from assignment1 import emails_between
from email.parser import Parser
from email.message import EmailMessage
# red_flags = {
# "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
# "skilling-j": ["skilling@enron.com", "jeff.skilling@enron.com"]
# }
# res=emails_between(red_flags)
# print(len(res))
# for r in res:
#     print("Date:",r['Date'],
#           "\nFrom:",r["From"],
#           "\n","To:",r['To'],
#           "\nSubject",r['Subject'],
#           "\nCC",r['Cc'],
#           "\nBCC",r['Bcc'],
#           "\nMessage",r.get_content())
# Example usage
suspects = {
    "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
    "skilling-j": ["jeff.skilling@enron.com", "jeffrey.skilling@enron.com"]
}

emails = emails_between(suspects)
print(f"Found {len(emails)} emails between the suspects.")


ModuleNotFoundError: No module named 'assignment1'

In [2]:
import os
from email.parser import Parser
# from typing import List, Dict

def emails_between(suspects):
    emails_between = []
    email_directory = "enron_email_corpus"
    
    parser = Parser()

    for root, dirs, files in os.walk(email_directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)

            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                    msg = parser.parse(file)

                sender = msg.get('From')
                recipients = msg.get('To')

                if sender and recipients:
                    recipient_emails = set([email.strip() for email in recipients.split(',') if email.strip() != ''])
                    
                    # Check if the sender is a suspect
                    sender_match = any(sender.lower() == address.lower() for account, addresses in suspects.items() for address in addresses)
                    
                    # Check if any recipients are suspects and limit to 20 or fewer unique recipients
                    recipient_matches = any(recipient.lower() in sum(suspects.values(), []) for recipient in recipient_emails)
                    
                    if sender_match and recipient_matches and len(recipient_emails) <= 20:
                        email_detail = {'From': sender, 'To': list(recipient_emails), 'Message': msg}
                        emails_between.append(email_detail)
            except Exception as e:
                print(f"Failed to parse {file_path}: {e}")
                continue

    return emails_between

# Example usage:
# red_flags = {
#     "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
#     "skilling-j": ["skilling@enron.com", "jeff.skilling@enron.com"]
# }

# emails = emails_between(red_flags)
# for email in emails[:10]:  # Print details of the first 10 emails to check
#     print(f"From: {email['From']}, To: {email['To']}, Message ID: {email['Message'].get('Message-ID')}")

# print(f"Found {len(emails)} emails between suspects.")
red_flags = {
"lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
"skilling-j": ["skilling@enron.com", "jeff.skilling@enron.com"]
}
res=emails_between(red_flags)
print(res)
for r in res:
    print("Date:",r['Date'],
          "\nFrom:",r["From"],
          "\n","To:",r['To'],
          "\nSubject",r['Subject'],
          "\nCC",r['Cc'],
          "\nBCC",r['Bcc'],
          "\nMessage",r.get_content())

[{'From': 'skilling@enron.com', 'To': ['jeff.skilling@enron.com'], 'Message': <email.message.Message object at 0x1069d8e10>}]


KeyError: 'Date'

In [ ]:
import os
from email.parser import Parser

def emails_between(suspects):
    emails_between = []
    email_directory = "enron_email_corpus"
    
    parser = Parser()

    # Pre-process suspects for efficient lookup
    all_suspects = {email.lower(): account for account, emails in suspects.items() for email in emails}

    for root, dirs, files in os.walk(email_directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)

            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                    msg = parser.parse(file)

                sender = msg.get('From')
                recipients = msg.get('To')

                if sender and recipients:
                    # Process recipients to ensure lowercase and strip spaces
                    processed_recipients = [email.strip().lower() for email in recipients.split(',') if email.strip()]

                    # Optimized check without using set and any
                    sender_match = sender.lower() in all_suspects
                    recipient_matches = False
                    unique_recipients = []
                    for recipient in processed_recipients:
                        if recipient in all_suspects:
                            recipient_matches = True
                        if recipient not in unique_recipients:
                            unique_recipients.append(recipient)

                    if sender_match and recipient_matches and len(unique_recipients) <= 20:
                        emails_between.append(msg)
            except Exception as e:
                print(f"Failed to parse {file_path}: {e}")
                continue

    return emails_between

# Example usage:
red_flags = {
    "lay-k": ["kenneth.lay@enron.com", "klay@enron.com"],
    "skilling-j": ["skilling@enron.com", "jeff.skilling@enron.com"]
}

emails = emails_between(red_flags)
print(f"Found {len(emails)} emails between suspects.")
